# Installing Marlowe CLI and Associated Tools

This lecture describes how to install `marlowe-cli` and the other tools needed to run Marlowe contracts.

## Cloning the Git Repository for Marlowe

The Marlowe repository for Cardano is located at <<https://github.com/input-output-hk/marlowe-cardano.git>>.
*   This repository contains the Marlowe source code, documentation, examples, and test cases.
*   The repository is needed if you want to use Marlowe from a Nix shell.
*   It is convenient, but not strictly required, if you install Marlowe tools from the Marlowe Pioneers Daedalus installation.

The following `git` command will copy the repository onto your machine.

In [1]:
git clone git@github.com:input-output-hk/marlowe-cardano.git -b mpp-cli-lectures

Cloning into 'marlowe-cardano'...
remote: Enumerating objects: 127773, done.
remote: Counting objects: 100% (546/546), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 127773 (delta 251), reused 513 (delta 234), pack-reused 127227
Receiving objects: 100% (127773/127773), 118.65 MiB | 33.29 MiB/s, done.
Resolving deltas: 100% (81755/81755), done.


## Installing and Running Marlowe CLI

The Marlowe CLI tool is written in the Haskell language. Here are four methods for installing `marlowe-cli`, listed in order of increasing difficulty.
1.  *Via Daedalus:* A copy of `marlowe-cli` is provided with the Daedalus wallet for Marlowe Pioneers.
2.  *Via Nix Shell:* This requires running either NixOs or the Nix package manager. See <<https://nixos.org/>>.
3.  *Via Nix:* This also requires NixOs or the Nix package manager.
4.  *Via Cabal:* This requires installing the Haskell tookchain. See <<https://www.haskell.org/downloads/>>.

In this lecture, we only discuss the first and second installation methods. The Marlowe documentation at <<https://github.com/input-output-hk/marlowe-cardano/blob/main/README.adoc>> provides instructions for the third and fourth installation methods.

👉 Installation assistance is available on the Marlowe Discord channel <<https://discord.com/channels/826816523368005654/936295815926927390/936316494042779698>>.

## Installation via Daedalus

***TO BE WRITTEN***

*   marlowe-cli
*   cardano-cli
*   cardano-wallet
*   cardano-node
*   CARDANO_NODE_SOCKET_PATH

👉 Installation assistance is available on the Marlowe Discord channel <<https://discord.com/channels/826816523368005654/936295815926927390/936316494042779698>>.

## Installation via the Nix Shell

NixOS and the Nix package manager are available at <<https://nixos.org/>>.
 
👉 See <<https://github.com/input-output-hk/marlowe-cardano/blob/main/README.adoc#how-to-set-up-the-iohk-binary-caches>> on how to set up binary caches for the Nix build. This will greatly speed the build process.

Once the binary caches have been set up, simply enter a nix shell.

In [5]:
nix-shell marlowe-cardano/shell.nix

The `marlowe-cli` and the other required executable programs are all available within that shell.

In [5]:
marlowe-cli --version

marlowe-cli 0.0.3.1 @ 


In [6]:
cardano-cli --version

cardano-cli 1.34.0 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [7]:
cardano-wallet version

v2022-01-18 (git revision: 9d5ab5ea5e51b4fab7e77cd1af6bc00822fe9d77)


In [8]:
cardano-node --version

cardano-node 1.34.1 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


👉 Installation assistance is available on the Marlowe Discord channel <<https://discord.com/channels/826816523368005654/936295815926927390/936316494042779698>>.

## Creating Payment and Signing Keys

***TO BE WRITTEN***

## Starting the Cardano Node

***TO BE WRITTEN***

## Other Lectures

*   Overview of Marlowe CLI
*   ~Installing Marlowe CLI and Associated Tools~
*   Running Marlowe Contracts without Blockchain Transactions
*   Running Marlowe Contacts on the Blockchain
*   Running Marlowe Contracts with the Plutus Application Backend (PAB)
*   Reference for Marlowe CLI Commands